In [4]:
import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [9]:
#Createing the connection to the database part 1
mongo = MongoClient(port=27017)
db = mongo['hockey_model']
db.list_collection_names()

['2022-2023', '2023-2023']

In [12]:
#creating connections part 2: putting it into a dataframe
data_2022_2023 = db['2022-2023']
answer = list(data_2022_2023.find({}))
df_2022 = pd.json_normalize(answer)

In [13]:
df_2022.head()

,_id,,playerId,season,name,team,position,situation,games_played,icetime,...,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,Line_or_Pair
0,659df24b1824a1bb36f2b0aa,0,8471817,2022,Ryan Reaves,MIN,R,all,73,40825,...,674.73,491,489.13,4.67,6.20,6.20,27.06,26.92,26.43,3rd line
1,659df24b1824a1bb36f2b0ab,1,8480950,2022,Ilya Lyubushkin,BUF,D,all,68,61236,...,1104.67,863,864.75,10.65,15.01,14.72,62.63,62.82,60.81,3rd pair
2,659df24b1824a1bb36f2b0ac,2,8475625,2022,Matt Irwin,WSH,D,all,61,47720,...,791.60,562,565.64,5.69,5.62,5.62,34.16,34.30,33.73,3rd pair
3,659df24b1824a1bb36f2b0ad,3,8480860,2022,Kevin Bahl,NJD,D,all,42,35331,...,508.74,378,379.33,3.76,7.03,7.03,21.02,21.10,20.84,3rd pair
4,659df24b1824a1bb36f2b0ae,4,8477952,2022,Robby Fabbri,DET,C,all,28,26877,...,425.03,316,313.93,3.19,2.47,2.47,20.77,20.86,20.65,2nd line
